In [ ]:
# This Colab requires TF 2.5.
!pip install -U tensorflow>=2.5

In [ ]:
import glob, os
import pathlib
import requests
import matplotlib
import matplotlib.pyplot as plt
import io
import scipy.misc
import pandas as pd
import numpy as np
import json
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen
import tensorflow as tf
import tensorflow_hub as hub
import cv2     # for capturing videos
import math   # for mathematical operations
from keras.preprocessing import image   # for preprocessing the images
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images

tf.get_logger().setLevel('ERROR')

In [ ]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2683, done.
remote: Counting objects: 100% (2683/2683), done.
remote: Compressing objects: 100% (2230/2230), done.
remote: Total 2683 (delta 681), reused 1277 (delta 420), pack-reused 0
Receiving objects: 100% (2683/2683), 32.65 MiB | 28.36 MiB/s, done.
Resolving deltas: 100% (681/681), done.


In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Reading package lists...
Building dependency tree...
Reading state information...
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1652208 sha256=ebfcf537baaf2eeb65192781537f1a45e1fee9867d696eb6a34f815bb5b8b204
  Stored in directory: /tmp/pip-ephem-wheel-cache-sb1ms0ra/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.2-cp37-none-any.whl size=44011 sha256=87ab48cf8bb9f04baf1d7e080fd6ea222126d55de33248b2c909fae70653c174
  Stored in directory: /root/.cache/pip/wheels/ee/ee/18/c466221ca6900e3efce2f4ea9c329288808679aecdcb2838d3
  Created wheel for dill: filename=dill-0.3.1.1-cp37-none-any.whl size=78545 sha256=528ed41dbb80e24fd6efbe8fa671b1e9ba2eda38c4cf8dd625c994cdef5f2103
  Stored in directory: /root/.cac



ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.29.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.2 which is incompatible.


In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

In [ ]:
model = {
'Faster R-CNN ResNet50 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1'
}

In [ ]:
COCO17_HUMAN_POSE_KEYPOINTS = [(0, 1),
 (0, 2),
 (1, 3),
 (2, 4),
 (0, 5),
 (0, 6),
 (5, 7),
 (7, 9),
 (6, 8),
 (8, 10),
 (5, 6),
 (5, 11),
 (6, 12),
 (11, 12),
 (11, 13),
 (13, 15),
 (12, 14),
 (14, 16)]

In [ ]:
PATH_TO_LABELS = './models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
def load_model():
  return hub.load(model['Faster R-CNN ResNet50 V1 640x640'])
   

In [ ]:
def load_image_into_numpy_array(path):
  image = None
  if(path.startswith('http')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))

  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)

##MAIN PROGRAM STARTS HERE

In [ ]:
def main():
  count = 0
  videoFile = "cobavideo.mp4"
  cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
  frameRate = cap.get(5) #frame rate
  x=1
  while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="frame%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
  cap.release()
  os.chdir(os.getcwd())
  frames = []
  for file in glob.glob("*.jpg"):
    print(file)
    frames.append(file)
  for i in range(len(frames)):
    image_path = os.path.abspath(frames[i])
    image_np = load_image_into_numpy_array(image_path)
    results = load_model()(image_np)
    result = {key:value.numpy() for key,value in results.items()}
    array_1 = pd.DataFrame(result['detection_classes'][0], columns=['Class'])
    array_2 = pd.DataFrame(result['detection_scores'][0],columns=['Score'])
    df = pd.concat([array_1, array_2], axis=1)
    df2 = df[(df['Class']==1.0) & (df['Score']>=0.1)]
    payload = {'total_penumpang' : len(df2)}
    requests.put('http://34.87.79.10/api/buses/1', data=payload)

In [ ]:
main()

frame169.jpg
frame162.jpg
frame26.jpg
frame1.jpg
frame41.jpg
frame138.jpg
frame106.jpg
frame84.jpg
frame144.jpg
frame102.jpg
frame34.jpg
frame31.jpg
frame118.jpg
frame49.jpg
frame62.jpg
frame91.jpg
frame18.jpg
frame130.jpg
frame27.jpg
frame108.jpg
frame77.jpg
frame35.jpg
frame165.jpg
frame50.jpg
frame15.jpg
frame29.jpg
frame96.jpg
frame6.jpg
frame66.jpg
frame92.jpg
frame16.jpg
frame39.jpg
frame127.jpg
frame3.jpg
frame114.jpg
frame119.jpg
frame146.jpg
frame128.jpg
frame112.jpg
frame81.jpg
frame88.jpg
frame60.jpg
frame43.jpg
frame95.jpg
frame33.jpg
frame105.jpg
frame163.jpg
frame72.jpg
frame78.jpg
frame93.jpg
frame48.jpg
frame115.jpg
frame54.jpg
frame126.jpg
frame116.jpg
frame121.jpg
frame133.jpg
frame71.jpg
frame160.jpg
frame148.jpg
frame147.jpg
frame80.jpg
frame158.jpg
frame55.jpg
frame86.jpg
frame24.jpg
frame14.jpg
frame12.jpg
frame40.jpg
frame125.jpg
frame153.jpg
frame28.jpg
frame19.jpg
frame122.jpg
frame46.jpg
frame56.jpg
frame94.jpg
frame76.jpg
frame10.jpg
frame47.jpg
frame69.jpg
f